# Time Series Forecasting

This notebook demonstrates the new modular architecture for time series forecasting experiments.

## Features:
- 🔧 Configuration-driven experiments
- 📊 Unified logging and metrics
- 🎨 Interactive visualizations
- 🔄 Rolling window validation
- ⚡ Parallel model execution

Cel 1 - import

In [1]:
import sys
from pathlib import Path
import logging
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
import time
import sqlite3
import hashlib
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')
logging.getLogger('build_training_set').setLevel(logging.ERROR)

current_dir = Path.cwd()
if "ENEXIS" in str(current_dir):
    while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
        current_dir = current_dir.parent
    project_root = current_dir
else:
    project_root = current_dir

src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from utils.build_training_set import build_training_set

training_data = build_training_set(
    train_start="2025-01-01 00:00:00",
    train_end="2025-03-14 23:00:00",
    run_date="2025-03-15 12:00:00"
)

training_data = training_data.set_index('target_datetime')
training_data.index = pd.to_datetime(training_data.index, utc=True)

print(f"Dataset loaded: {training_data.shape[0]} rows, {training_data.shape[1]} columns")
print(f"Date range: {training_data.index.min().date()} to {training_data.index.max().date()}")
print(f"Price range: {training_data['Price'].min():.4f} to {training_data['Price'].max():.4f}")
print(f"Data completeness: {(1 - training_data['Price'].isna().sum() / len(training_data)) * 100:.1f}%")

Dataset loaded: 1752 rows, 19 columns
Date range: 2025-01-01 to 2025-03-14
Price range: -0.0204 to 0.5235
Data completeness: 100.0%


cell 2 - model config

In [2]:
EXOG_VARS = [
    'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 
    'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 
    'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 
    'weekday_sin', 'hour_sin', 'weekday_cos'
]

available_exog = [col for col in EXOG_VARS if col in training_data.columns]

config_file = project_root / "src" / "config" / "best_sarimax_params.json"
if config_file.exists():
    with open(config_file, 'r') as f:
        best_params = json.load(f)
    current_order = tuple(best_params['order'])
    current_seasonal = tuple(best_params['seasonal_order'])
    print(f"Using optimized parameters: order={current_order}, seasonal={current_seasonal}")
    print(f"Expected improvement: {best_params.get('improvement_vs_baseline', 0):.1f}%")
else:
    current_order = (1, 0, 1)
    current_seasonal = (1, 1, 1, 24)
    print(f"Using default parameters: order={current_order}, seasonal={current_seasonal}")

print(f"Exogenous variables available: {len(available_exog)}/{len(EXOG_VARS)}")

Using optimized parameters: order=(2, 0, 0), seasonal=(1, 1, 0, 24)
Expected improvement: 30.1%
Exogenous variables available: 18/18


Cell 3 - 30 day rolling window

In [3]:
from utils.validation_utils import run_validation_experiment

print("Running 30-day rolling window validation...")

start_time = time.time()
results_df = run_validation_experiment(training_data, EXOG_VARS, n_days=30)
elapsed_time = time.time() - start_time

for _, row in results_df.iterrows():
    day = int(row['Day'])
    test_date = row['Test_Date']
    
    if row.get('Status') in ['SPLIT_FAIL', 'LOAD_FAIL']:
        status_str = row['Status']
    else:
        naive_str = f"N:{row['Naive']:.4f}" if not pd.isna(row['Naive']) else "N:FAIL"
        sarima_str = f"S:{row['SARIMA']:.4f}" if not pd.isna(row['SARIMA']) else "S:FAIL"
        if row.get('SARIMA_Fallback'):
            sarima_str += "*"
        sarimax_str = f"X:{row['SARIMAX']:.4f}" if not pd.isna(row['SARIMAX']) else "X:FAIL"
        status_str = f"{naive_str} | {sarima_str} | {sarimax_str}"
    
    print(f"Day {day:2d}: {test_date} | {status_str}")

print(f"\nValidation completed in {elapsed_time:.1f} seconds")

Running 30-day rolling window validation...
Day  1: 2025-03-15 | N:0.0211 | S:0.0190* | X:0.0152
Day  2: 2025-03-16 | N:0.0211 | S:0.0190* | X:0.0150
Day  3: 2025-03-17 | N:0.0197 | S:0.0185* | X:0.0157
Day  4: 2025-03-18 | N:0.0215 | S:0.0187* | X:0.0146
Day  5: 2025-03-19 | N:0.0243 | S:0.0202* | X:0.0156
Day  6: 2025-03-20 | N:0.0189 | S:0.0191* | X:0.0159
Day  7: 2025-03-21 | N:0.0222 | S:0.0190* | X:0.0177
Day  8: 2025-03-22 | N:0.0290 | S:0.0196* | X:0.0170
Day  9: 2025-03-23 | N:0.0247 | S:0.0210* | X:0.0166
Day 10: 2025-03-24 | N:0.0231 | S:0.0179* | X:0.0164
Day 11: 2025-03-25 | N:0.0194 | S:0.0175* | X:0.0175
Day 12: 2025-03-26 | N:0.0252 | S:0.0224* | X:0.0220
Day 13: 2025-03-27 | N:0.0278 | S:0.0187* | X:0.0166
Day 14: 2025-03-28 | N:0.0196 | S:0.0193* | X:0.0228
Day 15: 2025-03-29 | N:0.0344 | S:0.0227* | X:0.0248
Day 16: 2025-03-30 | N:0.0267 | S:0.0235* | X:0.0227
Day 17: 2025-03-31 | N:0.0317 | S:0.0253* | X:0.0204
Day 18: 2025-04-01 | N:0.0258 | S:0.0236* | X:0.0238
Da

cell 4 - performance

In [4]:
from utils.validation_utils import analyze_feature_contributions, generate_validation_summary

feature_importance, aic, bic = analyze_feature_contributions(training_data, EXOG_VARS)
summary = generate_validation_summary(results_df, feature_importance, aic, bic, EXOG_VARS)

print("Model Performance Summary:")
for model, stats in summary['performance'].items():
    print(f"{model:8s}: Mean={stats['mean']:.6f} | Std={stats['std']:.6f} | Range=[{stats['min']:.6f}, {stats['max']:.6f}]")

print(f"\nModel Improvements:")
improvements = summary['improvements']
print(f"SARIMA vs Naive: {improvements['sarima_vs_naive']:.1f}% improvement")
print(f"SARIMAX vs Naive: {improvements['sarimax_vs_naive']:.1f}% improvement") 
print(f"SARIMAX vs SARIMA: {improvements['sarimax_vs_sarima']:.1f}% improvement")

if feature_importance:
    print(f"\nTop 10 Most Important Features:")
    for i, (var, coef) in enumerate(feature_importance[:10], 1):
        print(f"  {i:2d}. {var:<25} | Coefficient: {coef:.6f}")
    
    print(f"\nFeature Concentration:")
    print(f"Top 5 features account for {summary['feature_concentration']:.1f}% of total impact")

print(f"\nModel Quality Metrics:")
if aic and bic:
    print(f"SARIMAX AIC: {aic:.2f} | BIC: {bic:.2f}")

print(f"\nRecommendation: {summary['best_model']} is the best performing model")

Model Performance Summary:
Naive   : Mean=0.027800 | Std=0.008388 | Range=[0.017342, 0.053230]
SARIMA  : Mean=0.023933 | Std=0.006239 | Range=[0.017226, 0.039747]
SARIMAX : Mean=0.021836 | Std=0.006040 | Range=[0.014638, 0.036371]

Model Improvements:
SARIMA vs Naive: 13.9% improvement
SARIMAX vs Naive: 21.5% improvement
SARIMAX vs SARIMA: 8.8% improvement

Top 10 Most Important Features:
   1. hour_sin                  | Coefficient: 0.847852
   2. weekday_cos               | Coefficient: 0.261831
   3. diffuse_radiation         | Coefficient: 0.092824
   4. is_non_working_day        | Coefficient: 0.066030
   5. is_dst                    | Coefficient: 0.048277
   6. month                     | Coefficient: 0.034975
   7. yearday_sin               | Coefficient: 0.008029
   8. is_weekend                | Coefficient: 0.005488
   9. cloud_cover               | Coefficient: 0.004569
  10. yearday_cos               | Coefficient: 0.002627

Feature Concentration:
Top 5 features account f

In [ ]:
def get_data_hash(training_data):
    data_str = f"{training_data.shape}_{training_data.iloc[0, 0]}_{training_data.iloc[-1, 0]}"
    return hashlib.md5(data_str.encode()).hexdigest()[:8]

def check_existing_results(order, seasonal_order, log_db):
    conn = sqlite3.connect(log_db)
    cursor = conn.cursor()
    cursor.execute("""
        SELECT mean_rmse, improvement_vs_baseline, overall_score, created_at
        FROM arima_validation_results 
        WHERE order_params = ? AND seasonal_order_params = ?
        AND created_at > datetime('now', '-7 days')
        ORDER BY created_at DESC LIMIT 1
    """, (str(order), str(seasonal_order)))
    result = cursor.fetchone()
    conn.close()
    return result

def test_parameter_fast(order, seasonal_order, training_data, exog_vars, baseline_rmse):
    from utils.validation_utils import run_validation_experiment
    import utils.validation_utils as val_utils
    
    original_validation = val_utils.run_single_day_validation
    
    def modified_validation(day, training_data, exog_vars):
        train_start_date = datetime(2025, 1, 1) + timedelta(days=day)
        run_date = datetime(2025, 3, 15) + timedelta(days=day)
        
        try:
            if day == 0:
                daily_data = training_data.copy()
            else:
                daily_data = training_data.copy()
                np.random.seed(day)
                noise_factor = 0.001 * day
                daily_data['Price'] = daily_data['Price'] + np.random.normal(0, noise_factor, len(daily_data))
            
            split_point = daily_data.index[-24]
            train_data = daily_data[daily_data.index < split_point]['Price'].copy()
            test_data = daily_data[daily_data.index >= split_point]['Price'].copy()
            
            day_results = {
                'Day': day + 1,
                'Test_Date': run_date.strftime('%Y-%m-%d'),
                'Train_Samples': len(train_data),
                'Test_Samples': len(test_data)
            }
            
            if exog_vars:
                train_exog = daily_data[daily_data.index < split_point][exog_vars].copy()
                test_exog = daily_data[daily_data.index >= split_point][exog_vars].copy()
                
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    model = SARIMAX(
                        train_data, exog=train_exog, order=order, seasonal_order=seasonal_order,
                        enforce_stationarity=False, enforce_invertibility=False
                    )
                    fitted_model = model.fit(method='lbfgs', maxiter=15, disp=False)
                    forecast = fitted_model.forecast(steps=len(test_data), exog=test_exog)
                    rmse = np.sqrt(mean_squared_error(test_data, forecast))
                    day_results['SARIMAX'] = rmse
            
            return day_results
        except Exception:
            return {'Day': day + 1, 'Status': 'FAIL'}
    
    val_utils.run_single_day_validation = modified_validation
    
    try:
        results_df = val_utils.run_validation_experiment(training_data, exog_vars, n_days=5)
        valid_results = results_df['SARIMAX'].dropna()
        
        if len(valid_results) > 0:
            mean_rmse = valid_results.mean()
            std_rmse = valid_results.std()
            improvement = ((baseline_rmse - mean_rmse) / baseline_rmse) * 100
            score = 1 / (1 + mean_rmse * 100) + 1 / (1 + std_rmse * 1000)
            
            return {
                'order': order,
                'seasonal_order': seasonal_order,
                'mean_rmse': mean_rmse,
                'std_rmse': std_rmse,
                'improvement_vs_baseline': improvement,
                'overall_score': score,
                'success': True
            }
    finally:
        val_utils.run_single_day_validation = original_validation
    
    return {'success': False}

def run_optimization():
    print("Starting Auto-ARIMA optimization...")
    
    log_db = project_root / "src" / "data" / "logs.db"
    log_db.parent.mkdir(parents=True, exist_ok=True)
    
    conn = sqlite3.connect(log_db)
    conn.execute("""
        CREATE TABLE IF NOT EXISTS arima_validation_results (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            order_params TEXT, seasonal_order_params TEXT, mean_rmse REAL, std_rmse REAL,
            improvement_vs_baseline REAL, overall_score REAL, created_at TEXT
        )
    """)
    conn.commit()
    conn.close()
    
    current_best_rmse = summary['performance']['SARIMAX']['mean']
    
    parameter_combinations = [
        ((1, 0, 1), (1, 1, 1, 24)),
        ((2, 0, 1), (1, 1, 1, 24)),
        ((1, 0, 2), (1, 1, 1, 24)),
        ((2, 0, 0), (1, 1, 0, 24)),
        ((1, 1, 1), (1, 1, 1, 24)),
        ((2, 0, 2), (0, 1, 0, 24))
    ]
    
    best_config = None
    tested_count = 0
    cached_count = 0
    
    for order, seasonal in parameter_combinations:
        cached_result = check_existing_results(order, seasonal, log_db)
        
        if cached_result:
            cached_count += 1
            mean_rmse, improvement, score, created_at = cached_result
            print(f"📦 CACHED {order}, {seasonal}: RMSE={mean_rmse:.6f}, Improvement={improvement:.1f}%")
            
            if not best_config or score > best_config['overall_score']:
                best_config = {
                    'order': order, 'seasonal_order': seasonal, 'mean_rmse': mean_rmse,
                    'improvement_vs_baseline': improvement, 'overall_score': score
                }
        else:
            tested_count += 1
            print(f"🧪 TESTING {order}, {seasonal}...")
            
            result = test_parameter_fast(order, seasonal, training_data, EXOG_VARS, current_best_rmse)
            
            if result['success']:
                print(f"✅ RMSE={result['mean_rmse']:.6f}, Improvement={result['improvement_vs_baseline']:.1f}%")
                
                conn = sqlite3.connect(log_db)
                conn.execute("""
                    INSERT INTO arima_validation_results 
                    (order_params, seasonal_order_params, mean_rmse, std_rmse, 
                     improvement_vs_baseline, overall_score, created_at)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                """, (str(order), str(seasonal), result['mean_rmse'], result['std_rmse'],
                      result['improvement_vs_baseline'], result['overall_score'], 
                      datetime.utcnow().isoformat()))
                conn.commit()
                conn.close()
                
                if not best_config or result['overall_score'] > best_config['overall_score']:
                    best_config = result
            else:
                print("❌ Test failed")
    
    print(f"\nOptimization complete: {cached_count} cached, {tested_count} new tests")
    
    if best_config:
        config_file = project_root / "src" / "config" / "best_sarimax_params.json"
        config_file.parent.mkdir(parents=True, exist_ok=True)
        
        config_data = {
            'order': best_config['order'],
            'seasonal_order': best_config['seasonal_order'],
            'mean_rmse': best_config['mean_rmse'],
            'improvement_vs_baseline': best_config.get('improvement_vs_baseline', 0),
            'overall_score': best_config.get('overall_score', 0),
            'updated_at': datetime.utcnow().isoformat()
        }
        
        with open(config_file, 'w') as f:
            json.dump(config_data, f, indent=2)
        
        print(f"\nBest configuration found:")
        print(f"Order: {best_config['order']}")
        print(f"Seasonal: {best_config['seasonal_order']}")
        print(f"RMSE: {best_config['mean_rmse']:.6f}")
        print(f"Improvement: {best_config.get('improvement_vs_baseline', 0):.1f}%")
        print(f"Config saved to: {config_file}")
    
    return best_config

optimization_result = run_optimization()

Starting Auto-ARIMA optimization...
📦 CACHED (1, 0, 1), (1, 1, 1, 24): RMSE=0.021836, Improvement=21.5%
🧪 TESTING (2, 0, 1), (1, 1, 1, 24)...


In [ ]:
# ============================================================================
# FORCE FRESH OPTIMIZATION - Ignore cache and run everything fresh
# ============================================================================

def run_fresh_optimization(training_data, exog_vars, force_auto_arima=True):
    """Run optimization ignoring cache - for testing new approaches"""
    
    print("🔥 FORCING FRESH OPTIMIZATION (IGNORING CACHE)")
    print("="*60)
    print("⚠️  This will take 3-5 minutes and test everything fresh")
    
    from pathlib import Path
    import time
    
    PROJECT_ROOT = Path.cwd()
    if "ENEXIS" in str(PROJECT_ROOT):
        while PROJECT_ROOT.name != "ENEXIS" and PROJECT_ROOT.parent != PROJECT_ROOT:
            PROJECT_ROOT = PROJECT_ROOT.parent
    
    start_time = time.time()
    
    # Get baseline
    baseline_rmse = 0.025
    
    # Test fewer combinations but fresh
    parameter_combinations = [
        ((1, 0, 1), (1, 1, 1, 24)),  # Current standard
        ((2, 0, 1), (1, 1, 1, 24)),  # More AR
        ((1, 0, 2), (1, 1, 1, 24)),  # More MA
        ((1, 1, 1), (1, 1, 1, 24)),  # With differencing
        ((2, 0, 0), (1, 1, 0, 24)),  # AR only
    ]
    
    print(f"🧪 Testing {len(parameter_combinations)} fresh combinations...")
    
    results = []
    
    # Test each combination fresh (no cache lookup)
    for i, (order, seasonal) in enumerate(parameter_combinations, 1):
        print(f"\n🧪 {i}/{len(parameter_combinations)}: Testing {order}, {seasonal}")
        
        # Force fresh test by modifying the test function
        args = (order, seasonal, training_data, exog_vars, baseline_rmse, "FRESH")
        
        # Use our fast test but without cache checking
        try:
            from utils.validation_utils import run_validation_experiment
            import utils.validation_utils as val_utils
            
            # Modified validation for this specific test
            original_validation = val_utils.run_single_day_validation
            
            def test_validation(day, training_data, exog_vars):
                from datetime import datetime, timedelta
                import warnings
                from sklearn.metrics import mean_squared_error
                from statsmodels.tsa.statespace.sarimax import SARIMAX
                
                train_start_date = datetime(2025, 1, 1) + timedelta(days=day)
                run_date = datetime(2025, 3, 15) + timedelta(days=day)
                
                try:
                    if day == 0:
                        daily_data = training_data.copy()
                    else:
                        daily_data = training_data.copy()
                        np.random.seed(day)
                        noise_factor = 0.001 * day
                        daily_data['Price'] = daily_data['Price'] + np.random.normal(0, noise_factor, len(daily_data))
                    
                    split_point = daily_data.index[-24]
                    train_data = daily_data[daily_data.index < split_point]['Price'].copy()
                    test_data = daily_data[daily_data.index >= split_point]['Price'].copy()
                    
                    day_results = {
                        'Day': day + 1,
                        'Test_Date': run_date.strftime('%Y-%m-%d'),
                        'Train_Samples': len(train_data),
                        'Test_Samples': len(test_data)
                    }
                    
                    # Test with our specific parameters
                    if exog_vars:
                        train_exog = daily_data[daily_data.index < split_point][exog_vars].copy()
                        test_exog = daily_data[daily_data.index >= split_point][exog_vars].copy()
                        
                        with warnings.catch_warnings():
                            warnings.simplefilter("ignore")
                            model = SARIMAX(
                                train_data,
                                exog=train_exog,
                                order=order,
                                seasonal_order=seasonal,
                                enforce_stationarity=False,
                                enforce_invertibility=False
                            )
                            fitted_model = model.fit(method='lbfgs', maxiter=20, disp=False)
                            forecast = fitted_model.forecast(steps=len(test_data), exog=test_exog)
                            rmse = np.sqrt(mean_squared_error(test_data, forecast))
                            day_results['SARIMAX'] = rmse
                    
                    return day_results
                    
                except Exception:
                    return {'Day': day + 1, 'Status': 'FAIL'}
            
            val_utils.run_single_day_validation = test_validation
            
            test_start = time.time()
            results_df = val_utils.run_validation_experiment(training_data, exog_vars, n_days=7)  # 7 days
            test_time = time.time() - test_start
            
            val_utils.run_single_day_validation = original_validation
            
            valid_results = results_df['SARIMAX'].dropna()
            
            if len(valid_results) > 0:
                mean_rmse = valid_results.mean()
                std_rmse = valid_results.std()
                improvement = ((baseline_rmse - mean_rmse) / baseline_rmse) * 100
                score = 1 / (1 + mean_rmse * 100) + 1 / (1 + std_rmse * 1000)
                
                result = {
                    'order': order,
                    'seasonal_order': seasonal,
                    'mean_rmse': mean_rmse,
                    'std_rmse': std_rmse,
                    'improvement_vs_baseline': improvement,
                    'overall_score': score,
                    'test_time': test_time
                }
                
                results.append(result)
                print(f"✅ RMSE: {mean_rmse:.6f} | Improvement: {improvement:.1f}% | Time: {test_time:.1f}s")
            else:
                print("❌ Test failed")
                
        except Exception as e:
            print(f"❌ Error: {e}")
    
    # Auto-ARIMA phase
    if force_auto_arima:
        print(f"\n🤖 Running fresh Auto-ARIMA...")
        auto_start = time.time()
        
        try:
            from pmdarima import auto_arima
            
            split_point = training_data.index[-24]
            train_data = training_data[training_data.index < split_point]['Price'].dropna()
            
            model = auto_arima(
                train_data,
                seasonal=True,
                m=24,
                stepwise=True,
                max_p=2,
                max_q=2,
                max_P=1, 
                max_Q=1,
                suppress_warnings=True,
                error_action="warn"
            )
            
            auto_time = time.time() - auto_start
            auto_order = model.order
            auto_seasonal = model.seasonal_order
            
            print(f"🤖 Auto-ARIMA result: {auto_order}, {auto_seasonal} (took {auto_time:.1f}s)")
            
            # Test auto-ARIMA result
            print("🧪 Testing Auto-ARIMA suggestion...")
            # [Test auto-ARIMA the same way as above]
            
        except Exception as e:
            print(f"❌ Auto-ARIMA failed: {e}")
    
    total_time = time.time() - start_time
    
    print(f"\n🏆 FRESH OPTIMIZATION RESULTS:")
    print("="*50)
    
    if results:
        best_result = min(results, key=lambda x: x['mean_rmse'])
        
        print(f"Best Configuration:")
        print(f"  Order: {best_result['order']}")
        print(f"  Seasonal: {best_result['seasonal_order']}")
        print(f"  RMSE: {best_result['mean_rmse']:.6f}")
        print(f"  Improvement: {best_result['improvement_vs_baseline']:.1f}%")
        
        print(f"\nAll Results:")
        for r in sorted(results, key=lambda x: x['mean_rmse']):
            print(f"  {r['order']}, {r['seasonal_order']}: RMSE={r['mean_rmse']:.6f}, Time={r['test_time']:.1f}s")
    
    print(f"\n⏱️ Total fresh optimization time: {total_time:.1f} seconds")
    
    return results

# Run fresh optimization
print("Choose your optimization mode:")
print("1. 🔥 FRESH (ignore all cache, ~3-5 minutes)")  
print("2. ⚡ SMART (use cache where possible, ~30 seconds)")

mode = input("Enter 1 or 2: ").strip()

if mode == "1":
    fresh_results = run_fresh_optimization(training_data, EXOG_VARS, force_auto_arima=True)
else:
    print("Running smart optimization with cache...")
    # Run the previous cached version